In [1]:
import pandas as pd
import numpy as np
import requests
import os
pd.set_option('display.max_rows', 1000)
from bs4 import BeautifulSoup
import plotly as plt
%matplotlib inline
pd.set_option('display.max_columns', 100)

In [2]:
import chart_studio.plotly as py
import plotly.graph_objects as go

In [3]:
import chart_studio
chart_studio.tools.set_config_file(plotly_domain="https://plotly.com", plotly_api_domain="https://api.plotly.com")

## Analysis of Vehicles Sales in Spain 2018 & 2019

### Extract Data

In [4]:
url1 = 'http://www.aniacam.com/datos/'
url2 = '/archivos/automoviles/DossierAutomoviles_'
year = np.arange(2011, 2019,1)
year = year.astype('str')
month = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
xls = '.xls'

In [5]:
os.chdir('//Users//antoniosanchez//Documents//Data science//Projects//Car Sales in Spain//XLS')

for i in year:
    for j in month:
        url = url1+i+url2+j+i[2:]+xls
        title = ""
        try:
            excel_file = requests.get(url)
            title = excel_file.url.split('/')[-1:][0]
            open(title, 'wb').write(excel_file.content)
        except :
            print('There was a problem extracting the data at:'+ j+ i)

        
        

### Transform Data

In [6]:
def transform_df(df):
    month = df.iloc[2,1].split(' ')[1]
    year = df.iloc[2,1].split(' ')[2]
    #Actual columns
    columns = df.iloc[6, 1:5].values

    #Drop empty rows
    df.drop(np.arange(0,7,1), inplace=True)

    #Drop empty columns
    df.drop(df.columns[0], axis=1, inplace=True)
    df.drop(df.columns[4:], axis=1, inplace=True)

    #Reset columns and index
    df.columns = columns
    df.reset_index(drop=True, inplace=True)
    
    df.rename(columns={'Mes': 'Total'}, inplace= True)
    
    #Set month & Year for the given table
    df['Mes'] = month
    df['Year'] = year
    
    return (df)

In [7]:
sales_list = os.listdir()
aut_sales = pd.DataFrame(columns=['Marca', 'Segmento', 'Modelo',  'Mes', 'Year', 'Total'])
for i in sales_list:
    if(i.split('.')[1] != 'xls'):
        continue   
    raw_df = pd.read_excel(i, sheet_name=4)
    df = transform_df(raw_df)
    aut_sales = pd.concat([aut_sales,df.dropna()], sort=False)
    del raw_df, df
    
aut_sales.rename(columns={'Marca': 'Brand', 'Segmento':'Segment', 'Modelo': 'Model',  'Mes':'Month'}, inplace=True)    

Ensure that all electric vehicles are classified under **EV** segment. To ensure so a list with the most common EV in the market has been used.

In [8]:
EV = ['I-PACE', 'ZOE', 'KANGOO', 'E-MEHARI', 'C-ZERO', 'IONIQ', 'MONTERO iO', 'iMIEV', 'iOn', 'LEAF', 'MODEL X', 'MODEL S', 'i8', 'i3', 'ROADSTER']

In [9]:
aut_sales.loc[(aut_sales.Model.isin(EV))|(aut_sales.Segment=='Eléctricos'), 'Segment'] = 'EV'

Create a dataframe only for EV

In [10]:
ev =aut_sales[aut_sales.Segment=='EV']
ev.Year = ev.Year.astype(int)
ev.Total = ev.Total.astype(int)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


### Analysis

In [11]:
brands = ev.Brand.unique()
grp_brd = ev.groupby(['Brand', 'Year']).Total.sum()
data = list()
for i in brands:
    brnd = dict()
    amts = list()
    years = np.arange(2011, 2019,1)
    for j in years:
        if(j not in grp_brd[i].index):
            amts.append(0)
        else:
            amts.append(grp_brd[i][j])
    brnd['x'] = years
    brnd['y'] = amts
    brnd['name'] = i
    brnd['type'] = 'bar'
    data.append(brnd)


layout = {
  'xaxis': {'title': 'Year'},
  'yaxis': {'title': 'Number'},
  'barmode': 'relative',
  'title': 'EV Sales in Spain'
};
py.iplot({'data': data, 'layout': layout}, filename='barmode-relative')

#### Conclusions

IONIC 100% HIBRIDO ENTRA EN ESPANA EN ABRIL 2018, HACER AJUSTES CORRESPONDIENTES EN LA GRAFICA